In [1]:
import os
import sys
sys.path.append('/mnt/c/Users/44yos/RacePrediction/resnet_win5')
from data_setup.race_data.this_week_race_url import this_week_race_url
from data_setup.race_data.this_week_race_html import this_week_race_html
from data_setup.race_data.make_csv_from_this_week_html import make_csv_from_this_week_html
# from data_setup.race_data.make_csv_this_week_with_odds import make_csv_this_week_with_odds
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import os
from os import path

from pickle import load

from models import resnet
from tensorflow.python import keras
from tensorflow.python.keras.models import load_model
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 200)

In [2]:
this_week_race_url()

In [3]:
this_week_race_html()

In [4]:
os.makedirs('../data/csv/2022/Jun_4', exist_ok = True)
make_csv_from_this_week_html()

In [5]:
#make_csv_this_week_with_odds()

In [2]:
race_df = pd.read_csv("../data/csv/2022/Jun_4/6_26.csv", sep = ",")

In [3]:
print(len(race_df['race_id'].unique()))
print(race_df.shape)
print(race_df.head(200))

12
(168, 22)
             race_id race_round race_title    race_distance weather  \
0    202202010608&rf         8R  3歳以上1勝クラス     ダ1700m (右)\n       晴   
1    202202010608&rf         8R  3歳以上1勝クラス     ダ1700m (右)\n       晴   
2    202202010608&rf         8R  3歳以上1勝クラス     ダ1700m (右)\n       晴   
3    202202010608&rf         8R  3歳以上1勝クラス     ダ1700m (右)\n       晴   
4    202202010608&rf         8R  3歳以上1勝クラス     ダ1700m (右)\n       晴   
5    202202010608&rf         8R  3歳以上1勝クラス     ダ1700m (右)\n       晴   
6    202202010608&rf         8R  3歳以上1勝クラス     ダ1700m (右)\n       晴   
7    202202010608&rf         8R  3歳以上1勝クラス     ダ1700m (右)\n       晴   
8    202202010608&rf         8R  3歳以上1勝クラス     ダ1700m (右)\n       晴   
9    202202010608&rf         8R  3歳以上1勝クラス     ダ1700m (右)\n       晴   
10   202202010608&rf         8R  3歳以上1勝クラス     ダ1700m (右)\n       晴   
11   202202010608&rf         8R  3歳以上1勝クラス     ダ1700m (右)\n       晴   
12   202202010609&rf         9R       松前特別     芝2000m (右)\n     

In [4]:
odds = pd.DataFrame(columns = ["odds"])
odds["odds"] = [10.2,2.0,32.4,28.3,13.3,11.3,41.4,10.3,8.5,32.6,90.7,6.2,20.2,26.3,29.3,7.2,5.5,29.2,13.1,3.8,16.4,11.8,7.0,6.5,5.6,23.9,7.6,7.7,33.3,11.4,29.1,19.9,16.7,68.2,19.7,3.8,8.8,31.5,24.3,6.9,20.0,11.2,10.1,3.9,15.6,6.3,10.7,7.4,5.9,4.0,19.8,22.3,10.9,7.6,23.7,8.4,63.7,12.8,69.2,14.2,117.9,38.4,12.2,7.6,9.9,14.5,5.5,4.0,24.2,88.0,11.3,1.7,28.4,48.0,6.4,29.5,58.8,40.4,8.2,10.3,37.8,7.9,43.5,5.0,6.3,34.2,25.2,9.3,48.7,13.3,26.1,35.3,11.3,7.3,35.0,16.2,6.2,53.6,95.2,5.3,4.1,7.5,71.2,21.4,22.3,90.8,13.4,32.7,9.4,2.0,81.1,5.9,21.8,91.7,17.0,6.6,10.1,8.9,26.3,36.5,7.8,19.9,10.6,2.7,13.7,33.7,31.1,16.2,32.1,21.7,20.0,27.2,56.6,13.6,30.4,6.3,51.2,38.0,50.4,72.2,29.2,15.5,29.2,2.7,5.6,68.5,66.3,7.9,86.7,7.0,13.0,90.8,139.7,3.9,133.9,4.9,7.5,51.7,114.6,10.4,9.4,67.9,12.8,85.1,6.0,98.3,195.2,21.9]

In [5]:
print(odds.shape)
# print(odds)

(168, 1)


In [6]:
race_df["odds"] = odds["odds"]

In [7]:
race_df.tail(50)

,race_id,race_round,race_title,race_distance,weather,ground_condition,time,date,place,total_horse_number,order,frame_number,horse_number,horse_id,sex_and_age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight
118,202209030409&rf,9R,舞子特別,芝1400m (右)\n,晴,良,14:25発走,2022-6-26,阪神,17,?,1,2,2017106086,牝5,53.0,1180,?,?,?,26.3,?
119,202209030409&rf,9R,舞子特別,芝1400m (右)\n,晴,良,14:25発走,2022-6-26,阪神,17,?,2,3,2018106305,牝4,52.0,732,?,?,?,36.5,?
120,202209030409&rf,9R,舞子特別,芝1400m (右)\n,晴,良,14:25発走,2022-6-26,阪神,17,?,2,4,2019105449,牡3,53.0,1014,?,?,?,7.8,?
121,202209030409&rf,9R,舞子特別,芝1400m (右)\n,晴,良,14:25発走,2022-6-26,阪神,17,?,3,5,2016105622,牡6,55.0,1018,?,?,?,19.9,?
122,202209030409&rf,9R,舞子特別,芝1400m (右)\n,晴,良,14:25発走,2022-6-26,阪神,17,?,3,6,2018105373,牝4,53.0,1088,?,?,?,10.6,?
123,202209030409&rf,9R,舞子特別,芝1400m (右)\n,晴,良,14:25発走,2022-6-26,阪神,17,?,4,7,2019103782,牝3,51.0,1032,?,?,?,2.7,?
124,202209030409&rf,9R,舞子特別,芝1400m (右)\n,晴,良,14:25発走,2022-6-26,阪神,17,?,4,8,2018105263,牝4,53.0,1102,?,?,?,13.7,?
125,202209030409&rf,9R,舞子特別,芝1400m (右)\n,晴,良,14:25発走,2022-6-26,阪神,17,?,5,9,2018106391,牡4,53.0,1185,?,?,?,33.7,?
126,202209030409&rf,9R,舞子特別,芝1400m (右)\n,晴,良,14:25発走,2022-6-26,阪神,17,?,5,10,2017105413,牝5,51.0,1154,?,?,?,31.1,?
127,202209030409&rf,9R,舞子特別,芝1400m (右)\n,晴,良,14:25発走,2022-6-26,阪神,17,?,6,11,2016102102,牡6,55.0,5212,?,?,?,16.2,?


race_round

In [8]:
race_df['race_round'] = race_df['race_round'].str.strip('R \n')

In [9]:
race_df.head(5)

,race_id,race_round,race_title,race_distance,weather,ground_condition,time,date,place,total_horse_number,order,frame_number,horse_number,horse_id,sex_and_age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight
0,202202010608&rf,8,3歳以上1勝クラス,ダ1700m (右)\n,晴,良,13:35発走,2022-6-26,函館,12,?,1,1,2018100720,セ4,57.0,1030,?,?,?,10.2,?
1,202202010608&rf,8,3歳以上1勝クラス,ダ1700m (右)\n,晴,良,13:35発走,2022-6-26,函館,12,?,2,2,2019101236,牡3,54.0,1157,?,?,?,2.0,?
2,202202010608&rf,8,3歳以上1勝クラス,ダ1700m (右)\n,晴,良,13:35発走,2022-6-26,函館,12,?,3,3,2019103671,牡3,51.0,1200,?,?,?,32.4,?
3,202202010608&rf,8,3歳以上1勝クラス,ダ1700m (右)\n,晴,良,13:35発走,2022-6-26,函館,12,?,4,4,2019101536,牡3,54.0,1115,?,?,?,28.3,?
4,202202010608&rf,8,3歳以上1勝クラス,ダ1700m (右)\n,晴,良,13:35発走,2022-6-26,函館,12,?,5,5,2019105980,牡3,51.0,1177,?,?,?,13.3,?


race_rank

In [10]:
race_df["race_title"] = race_df["race_title"].replace('宝塚記念','宝塚記念G1')
# race_df["race_title"] = race_df["race_title"].replace('京都牝馬S','京都牝馬SG3')
# race_df["race_title"] = race_df["race_title"].replace('阪神C','阪神CG2')

In [11]:
race_rank = race_df["race_title"].str.extract('(G1|G2|G3)', expand = True)
race_rank.columns = {"race_rank"}

race_df = pd.concat([race_df, race_rank], axis = 1)
race_df.fillna(value = {'race_rank': 0}, inplace = True) # 元もデータに反映
race_df['race_rank'] = race_df['race_rank'].replace('G1', 3)
race_df['race_rank'] = race_df['race_rank'].replace('G2', 2)
race_df['race_rank'] = race_df['race_rank'].replace('G3', 1)

In [12]:
print("race_rank:", race_df["race_rank"].value_counts())

race_rank: 0    150
3     18
Name: race_rank, dtype: int64


In [13]:
race_df.head(5)

,race_id,race_round,race_title,race_distance,weather,ground_condition,time,date,place,total_horse_number,order,frame_number,horse_number,horse_id,sex_and_age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight,race_rank
0,202202010608&rf,8,3歳以上1勝クラス,ダ1700m (右)\n,晴,良,13:35発走,2022-6-26,函館,12,?,1,1,2018100720,セ4,57.0,1030,?,?,?,10.2,?,0
1,202202010608&rf,8,3歳以上1勝クラス,ダ1700m (右)\n,晴,良,13:35発走,2022-6-26,函館,12,?,2,2,2019101236,牡3,54.0,1157,?,?,?,2.0,?,0
2,202202010608&rf,8,3歳以上1勝クラス,ダ1700m (右)\n,晴,良,13:35発走,2022-6-26,函館,12,?,3,3,2019103671,牡3,51.0,1200,?,?,?,32.4,?,0
3,202202010608&rf,8,3歳以上1勝クラス,ダ1700m (右)\n,晴,良,13:35発走,2022-6-26,函館,12,?,4,4,2019101536,牡3,54.0,1115,?,?,?,28.3,?,0
4,202202010608&rf,8,3歳以上1勝クラス,ダ1700m (右)\n,晴,良,13:35発走,2022-6-26,函館,12,?,5,5,2019105980,牡3,51.0,1177,?,?,?,13.3,?,0


race_distance

In [14]:
race_df["race_distance"].unique()

array([' ダ1700m (右)\n', ' 芝2000m (右)\n', ' 芝1200m (右)\n', ' ダ2100m (左)\n',
       ' 芝1800m (左)\n', ' ダ1600m (左)\n', ' 芝1400m (左)\n',
       ' 芝1800m (右 外)\n', ' 芝1400m (右)\n', ' ダ1400m (右)\n',
       ' 芝2200m (右)\n'], dtype=object)

In [15]:
ground_type = race_df["race_distance"].str.extract('(芝|ダ)', expand = True)
circle = race_df["race_distance"].str.extract('(右|左)', expand = True)
distance = race_df["race_distance"].str.extract('(\d+)m', expand = True)

ground_type.columns = {"ground_type"}
circle.columns = {"circle"}
distance.columns = {"distance"}

race_df = pd.concat([race_df, ground_type], axis = 1)
race_df = pd.concat([race_df, circle], axis = 1)
race_df = pd.concat([race_df, distance], axis = 1)

race_df = pd.get_dummies(race_df, columns = ["ground_type", "circle"])

In [16]:
race_df["distance"] = race_df["distance"].astype(int)
race_df.drop(['race_distance'], axis = 1, inplace = True)

In [17]:
# # 存在しない右左回り適宜追加 ######### circle_左 or circle_右
add_circle = pd.DataFrame(columns = ["circle_右","circle_左"])
race_df = pd.concat([race_df,add_circle], axis = 1).fillna(0)

In [18]:
race_df.head(5)

,race_id,race_round,race_title,weather,ground_condition,time,date,place,total_horse_number,order,frame_number,horse_number,horse_id,sex_and_age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_左,circle_右,circle_左
0,202202010608&rf,8,3歳以上1勝クラス,晴,良,13:35発走,2022-6-26,函館,12,?,1,1,2018100720,セ4,57.0,1030,?,?,?,10.2,?,0,1700,1,0,1,0,0,0
1,202202010608&rf,8,3歳以上1勝クラス,晴,良,13:35発走,2022-6-26,函館,12,?,2,2,2019101236,牡3,54.0,1157,?,?,?,2.0,?,0,1700,1,0,1,0,0,0
2,202202010608&rf,8,3歳以上1勝クラス,晴,良,13:35発走,2022-6-26,函館,12,?,3,3,2019103671,牡3,51.0,1200,?,?,?,32.4,?,0,1700,1,0,1,0,0,0
3,202202010608&rf,8,3歳以上1勝クラス,晴,良,13:35発走,2022-6-26,函館,12,?,4,4,2019101536,牡3,54.0,1115,?,?,?,28.3,?,0,1700,1,0,1,0,0,0
4,202202010608&rf,8,3歳以上1勝クラス,晴,良,13:35発走,2022-6-26,函館,12,?,5,5,2019105980,牡3,51.0,1177,?,?,?,13.3,?,0,1700,1,0,1,0,0,0


weather

In [19]:
# race_df["weather"] = race_df["weather"].mask(race_df["place"] == '中山', '小雨')
# race_df["weather"] = race_df["weather"].mask(race_df["place"] == '中京', '晴')
#race_df["weather"] = race_df["weather"].mask(race_df["place"] == '札幌', '晴')

In [20]:
weather = race_df["weather"].str.extract('(晴|曇|小雨|雨|小雪|雪)', expand = True)
weather.columns = {"weather_circumstance"}
race_df = pd.concat([race_df, weather], axis = 1)
race_df = pd.get_dummies(race_df, columns = ["weather_circumstance"])
race_df.drop(['weather'], axis = 1, inplace = True)

In [21]:
race_df.head(5)

,race_id,race_round,race_title,ground_condition,time,date,place,total_horse_number,order,frame_number,horse_number,horse_id,sex_and_age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_左,circle_右,circle_左,weather_circumstance_晴
0,202202010608&rf,8,3歳以上1勝クラス,良,13:35発走,2022-6-26,函館,12,?,1,1,2018100720,セ4,57.0,1030,?,?,?,10.2,?,0,1700,1,0,1,0,0,0,1
1,202202010608&rf,8,3歳以上1勝クラス,良,13:35発走,2022-6-26,函館,12,?,2,2,2019101236,牡3,54.0,1157,?,?,?,2.0,?,0,1700,1,0,1,0,0,0,1
2,202202010608&rf,8,3歳以上1勝クラス,良,13:35発走,2022-6-26,函館,12,?,3,3,2019103671,牡3,51.0,1200,?,?,?,32.4,?,0,1700,1,0,1,0,0,0,1
3,202202010608&rf,8,3歳以上1勝クラス,良,13:35発走,2022-6-26,函館,12,?,4,4,2019101536,牡3,54.0,1115,?,?,?,28.3,?,0,1700,1,0,1,0,0,0,1
4,202202010608&rf,8,3歳以上1勝クラス,良,13:35発走,2022-6-26,函館,12,?,5,5,2019105980,牡3,51.0,1177,?,?,?,13.3,?,0,1700,1,0,1,0,0,0,1


In [22]:
# 存在しない天気適宜追加 #########
add_weather = pd.DataFrame(columns = ["weather_circumstance_小雨","weather_circumstance_雨","weather_circumstance_晴","weather_circumstance_曇","weather_circumstance_小雪","weather_circumstance_雪"])
race_df = pd.concat([race_df,add_weather], axis = 1).fillna(0)

In [23]:
race_df.head(5)

,race_id,race_round,race_title,ground_condition,time,date,place,total_horse_number,order,frame_number,horse_number,horse_id,sex_and_age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_左,circle_右,circle_左,weather_circumstance_晴,weather_circumstance_小雨,weather_circumstance_雨,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雪,weather_circumstance_雪
0,202202010608&rf,8,3歳以上1勝クラス,良,13:35発走,2022-6-26,函館,12,?,1,1,2018100720,セ4,57.0,1030,?,?,?,10.2,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0
1,202202010608&rf,8,3歳以上1勝クラス,良,13:35発走,2022-6-26,函館,12,?,2,2,2019101236,牡3,54.0,1157,?,?,?,2.0,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0
2,202202010608&rf,8,3歳以上1勝クラス,良,13:35発走,2022-6-26,函館,12,?,3,3,2019103671,牡3,51.0,1200,?,?,?,32.4,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0
3,202202010608&rf,8,3歳以上1勝クラス,良,13:35発走,2022-6-26,函館,12,?,4,4,2019101536,牡3,54.0,1115,?,?,?,28.3,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0
4,202202010608&rf,8,3歳以上1勝クラス,良,13:35発走,2022-6-26,函館,12,?,5,5,2019105980,牡3,51.0,1177,?,?,?,13.3,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0


ground_condition

In [24]:
# race_df["ground_condition"] = race_df["ground_condition"].mask(race_df["place"] == '中山', '稍重')
#race_df["ground_condition"] = race_df["ground_condition"].mask(race_df["place"] == '新潟', '良')
#race_df["ground_condition"] = "良"

In [25]:
race_df["ground_condition"] = race_df["ground_condition"].replace('(不良)', 4, regex = True)
race_df["ground_condition"] = race_df["ground_condition"].replace('(稍重)', 2, regex = True)
race_df["ground_condition"] = race_df["ground_condition"].replace('(重)', 3, regex = True)
race_df["ground_condition"] = race_df["ground_condition"].replace('(良)', 1, regex = True)

In [26]:
print("ground_condition:", race_df["ground_condition"].value_counts())

ground_condition: 1    168
Name: ground_condition, dtype: int64


In [27]:
race_df.head(5)

,race_id,race_round,race_title,ground_condition,time,date,place,total_horse_number,order,frame_number,horse_number,horse_id,sex_and_age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_左,circle_右,circle_左,weather_circumstance_晴,weather_circumstance_小雨,weather_circumstance_雨,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雪,weather_circumstance_雪
0,202202010608&rf,8,3歳以上1勝クラス,1,13:35発走,2022-6-26,函館,12,?,1,1,2018100720,セ4,57.0,1030,?,?,?,10.2,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0
1,202202010608&rf,8,3歳以上1勝クラス,1,13:35発走,2022-6-26,函館,12,?,2,2,2019101236,牡3,54.0,1157,?,?,?,2.0,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0
2,202202010608&rf,8,3歳以上1勝クラス,1,13:35発走,2022-6-26,函館,12,?,3,3,2019103671,牡3,51.0,1200,?,?,?,32.4,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0
3,202202010608&rf,8,3歳以上1勝クラス,1,13:35発走,2022-6-26,函館,12,?,4,4,2019101536,牡3,54.0,1115,?,?,?,28.3,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0
4,202202010608&rf,8,3歳以上1勝クラス,1,13:35発走,2022-6-26,函館,12,?,5,5,2019105980,牡3,51.0,1177,?,?,?,13.3,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0


datetime

In [28]:
race_df["date"] =  pd.to_datetime(race_df['date'])
race_df.drop(["time"], axis = 1, inplace = True)

In [29]:
race_df.head(5)

,race_id,race_round,race_title,ground_condition,date,place,total_horse_number,order,frame_number,horse_number,horse_id,sex_and_age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_左,circle_右,circle_左,weather_circumstance_晴,weather_circumstance_小雨,weather_circumstance_雨,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雪,weather_circumstance_雪
0,202202010608&rf,8,3歳以上1勝クラス,1,2022-06-26,函館,12,?,1,1,2018100720,セ4,57.0,1030,?,?,?,10.2,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0
1,202202010608&rf,8,3歳以上1勝クラス,1,2022-06-26,函館,12,?,2,2,2019101236,牡3,54.0,1157,?,?,?,2.0,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0
2,202202010608&rf,8,3歳以上1勝クラス,1,2022-06-26,函館,12,?,3,3,2019103671,牡3,51.0,1200,?,?,?,32.4,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0
3,202202010608&rf,8,3歳以上1勝クラス,1,2022-06-26,函館,12,?,4,4,2019101536,牡3,54.0,1115,?,?,?,28.3,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0
4,202202010608&rf,8,3歳以上1勝クラス,1,2022-06-26,函館,12,?,5,5,2019105980,牡3,51.0,1177,?,?,?,13.3,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0


place

In [30]:
race_df = pd.get_dummies(race_df, columns = ["place"])

In [31]:
race_df.head(5)

,race_id,race_round,race_title,ground_condition,date,total_horse_number,order,frame_number,horse_number,horse_id,sex_and_age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_左,circle_右,circle_左,weather_circumstance_晴,weather_circumstance_小雨,weather_circumstance_雨,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雪,weather_circumstance_雪,place_函館,place_東京,place_阪神
0,202202010608&rf,8,3歳以上1勝クラス,1,2022-06-26,12,?,1,1,2018100720,セ4,57.0,1030,?,?,?,10.2,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0
1,202202010608&rf,8,3歳以上1勝クラス,1,2022-06-26,12,?,2,2,2019101236,牡3,54.0,1157,?,?,?,2.0,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0
2,202202010608&rf,8,3歳以上1勝クラス,1,2022-06-26,12,?,3,3,2019103671,牡3,51.0,1200,?,?,?,32.4,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0
3,202202010608&rf,8,3歳以上1勝クラス,1,2022-06-26,12,?,4,4,2019101536,牡3,54.0,1115,?,?,?,28.3,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0
4,202202010608&rf,8,3歳以上1勝クラス,1,2022-06-26,12,?,5,5,2019105980,牡3,51.0,1177,?,?,?,13.3,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0


In [32]:
# 存在しない場所追加 '札幌', '中京', '新潟', '函館', '小倉', '京都','東京','福島'  '阪神' '中山'
add_place = pd.DataFrame(columns =['札幌','中京','新潟','小倉','京都','福島','中山'])
race_df = pd.concat([race_df,add_place], axis = 1).fillna(0)

In [33]:
race_df.head(5)

,race_id,race_round,race_title,ground_condition,date,total_horse_number,order,frame_number,horse_number,horse_id,sex_and_age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_左,circle_右,circle_左,weather_circumstance_晴,weather_circumstance_小雨,weather_circumstance_雨,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雪,weather_circumstance_雪,place_函館,place_東京,place_阪神,札幌,中京,新潟,小倉,京都,福島,中山
0,202202010608&rf,8,3歳以上1勝クラス,1,2022-06-26,12,?,1,1,2018100720,セ4,57.0,1030,?,?,?,10.2,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,202202010608&rf,8,3歳以上1勝クラス,1,2022-06-26,12,?,2,2,2019101236,牡3,54.0,1157,?,?,?,2.0,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,202202010608&rf,8,3歳以上1勝クラス,1,2022-06-26,12,?,3,3,2019103671,牡3,51.0,1200,?,?,?,32.4,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,202202010608&rf,8,3歳以上1勝クラス,1,2022-06-26,12,?,4,4,2019101536,牡3,54.0,1115,?,?,?,28.3,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,202202010608&rf,8,3歳以上1勝クラス,1,2022-06-26,12,?,5,5,2019105980,牡3,51.0,1177,?,?,?,13.3,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


sex_age

In [34]:
sex = race_df["sex_and_age"].str.extract('(牡|牝|セ)', expand = True)
sex.columns = {"sex"}
race_df = pd.concat([race_df, sex], axis = 1)
race_df = pd.get_dummies(race_df, columns = ["sex"])

race_df["sex_and_age"] = race_df["sex_and_age"].str.strip("牡牝セ\n")
race_df["sex_and_age"] = race_df["sex_and_age"].astype(int)
race_df = race_df.rename(columns = {"sex_and_age": "age"})

In [35]:
race_df.head(5)

,race_id,race_round,race_title,ground_condition,date,total_horse_number,order,frame_number,horse_number,horse_id,age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_左,circle_右,circle_左,weather_circumstance_晴,weather_circumstance_小雨,weather_circumstance_雨,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雪,weather_circumstance_雪,place_函館,place_東京,place_阪神,札幌,中京,新潟,小倉,京都,福島,中山,sex_セ,sex_牝,sex_牡
0,202202010608&rf,8,3歳以上1勝クラス,1,2022-06-26,12,?,1,1,2018100720,4,57.0,1030,?,?,?,10.2,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
1,202202010608&rf,8,3歳以上1勝クラス,1,2022-06-26,12,?,2,2,2019101236,3,54.0,1157,?,?,?,2.0,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
2,202202010608&rf,8,3歳以上1勝クラス,1,2022-06-26,12,?,3,3,2019103671,3,51.0,1200,?,?,?,32.4,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
3,202202010608&rf,8,3歳以上1勝クラス,1,2022-06-26,12,?,4,4,2019101536,3,54.0,1115,?,?,?,28.3,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
4,202202010608&rf,8,3歳以上1勝クラス,1,2022-06-26,12,?,5,5,2019105980,3,51.0,1177,?,?,?,13.3,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1


burden_weight

In [36]:
race_df["burden_weight"] = race_df["burden_weight"].astype(float)

odds

In [37]:
print(race_df["odds"].dtypes)

float64


In [38]:
race_df.to_csv("../data/csv/2022/Jun_4/refined_6_26_race.csv", index = False)

In [39]:
#race_df = pd.read_csv("../data/csv/2021/dec_4/refined_12_26_race.csv", sep = ",")

make_train_data

In [40]:
horse_df = pd.read_csv("../data/csv/horse_data/refined_horse_data.csv", sep = ",")
horse_f_df = pd.read_csv("../data/csv/horse_data/horse_featured_data.csv", sep = ",")

/home/yoshi/miniconda3/envs/tensorflow-gpu/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [41]:
race_df['order'] = race_df['order'].astype(str)
race_df['horse_id'] = race_df['horse_id'].astype(str)
race_df['jockey_id'] = race_df['jockey_id'].astype(str)
race_df['date'] = pd.to_datetime(race_df['date'])
print(race_df.shape)
print(race_df.dtypes)
race_df.head(3)

(168, 46)
race_id                            object
race_round                         object
race_title                         object
ground_condition                    int64
date                       datetime64[ns]
total_horse_number                  int64
order                              object
frame_number                        int64
horse_number                        int64
horse_id                           object
age                                 int64
burden_weight                     float64
jockey_id                          object
goal_time                          object
half_order                         object
last_time                          object
odds                              float64
horse_weight                       object
race_rank                           int64
distance                            int64
ground_type_ダ                       uint8
ground_type_芝                       uint8
circle_右                            uint8
circle_左                

,race_id,race_round,race_title,ground_condition,date,total_horse_number,order,frame_number,horse_number,horse_id,age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_左,circle_右,circle_左,weather_circumstance_晴,weather_circumstance_小雨,weather_circumstance_雨,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雪,weather_circumstance_雪,place_函館,place_東京,place_阪神,札幌,中京,新潟,小倉,京都,福島,中山,sex_セ,sex_牝,sex_牡
0,202202010608&rf,8,3歳以上1勝クラス,1,2022-06-26,12,?,1,1,2018100720,4,57.0,1030,?,?,?,10.2,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
1,202202010608&rf,8,3歳以上1勝クラス,1,2022-06-26,12,?,2,2,2019101236,3,54.0,1157,?,?,?,2.0,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
2,202202010608&rf,8,3歳以上1勝クラス,1,2022-06-26,12,?,3,3,2019103671,3,51.0,1200,?,?,?,32.4,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1


In [42]:
horse_df['order'] = horse_df['order'].astype(str)
horse_df['horse_id'] = horse_df['horse_id'].astype(str)
horse_df['date'] = pd.to_datetime(horse_df['date'])
print(horse_df.dtypes)
print(horse_df.shape)
horse_df.head(3)

horse_id                           object
father_id                          object
grandfather_id                     object
mother_id                          object
total_race_number                   int64
date                       datetime64[ns]
whole_horse_number                  int64
odds                              float64
order                              object
jockey_id                          object
burden_weight                     float64
race_distance                       int64
ground_condition                    int64
goal_time                         float64
half_order                        float64
last_time                         float64
horse_weight                      float64
weather_circumstance_小雨             int64
weather_circumstance_小雪             int64
weather_circumstance_晴              int64
weather_circumstance_曇              int64
weather_circumstance_雨              int64
weather_circumstance_雪              int64
main_place_その他                    

,horse_id,father_id,grandfather_id,mother_id,total_race_number,date,whole_horse_number,odds,order,jockey_id,burden_weight,race_distance,ground_condition,goal_time,half_order,last_time,horse_weight,weather_circumstance_小雨,weather_circumstance_小雪,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_雨,weather_circumstance_雪,main_place_その他,main_place_中京,main_place_中山,main_place_京都,main_place_函館,main_place_小倉,main_place_新潟,main_place_札幌,main_place_東京,main_place_福島,main_place_阪神,race_rank,ground_type_ダ,ground_type_芝,ground_type_障,horse_weight_dif
0,2019100026,000a011155,000a000178,2013106103,8,2022-06-18,16,28.9,5,732,54.0,1400,1,85.3,0.687500,36.6,478.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0.0
1,2019100026,000a011155,000a000178,2013106103,8,2022-05-22,13,12.0,4,732,56.0,1400,1,85.5,0.692308,37.8,478.0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,2.0
2,2019100026,000a011155,000a000178,2013106103,8,2022-01-10,15,3.8,1,1185,54.0,1400,1,86.0,0.200000,38.6,476.0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,6.0


In [43]:
horse_f_df['horse_id'] = horse_f_df['horse_id'].astype(str)
print(horse_f_df.dtypes)
print(horse_f_df.shape)
horse_f_df.head(3)

horse_id           object
grass_win_rate    float64
dart_win_rate     float64
win_rate          float64
dtype: object
(62073, 4)


,horse_id,grass_win_rate,dart_win_rate,win_rate
0,2019100026,0.0,0.125000,0.125000
1,2019100053,0.0,0.500000,0.500000
2,2019100055,0.0,0.333333,0.166667


In [44]:
pedigree_map = pd.read_csv("../data/csv/horse_data/pedigree_map.csv", sep = ",")
pedigree_map['horse_id'] = pedigree_map['horse_id'].astype(str)
print(pedigree_map.dtypes)

horse_id               object
f_grass_win_rate      float64
f_dart_win_rate       float64
f_win_rate            float64
g_f_grass_win_rate    float64
g_f_dart_win_rate     float64
g_f_win_rate          float64
m_grass_win_rate      float64
m_dart_win_rate       float64
m_win_rate            float64
dtype: object


add_pedigree_map to race_df

In [45]:
race_df = pd.merge(race_df, pedigree_map, on = "horse_id", how = "left").fillna(0)

In [46]:
race_df.head(5)

,race_id,race_round,race_title,ground_condition,date,total_horse_number,order,frame_number,horse_number,horse_id,age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_左,circle_右,circle_左,weather_circumstance_晴,weather_circumstance_小雨,weather_circumstance_雨,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雪,weather_circumstance_雪,place_函館,place_東京,place_阪神,札幌,中京,新潟,小倉,京都,福島,中山,sex_セ,sex_牝,sex_牡,f_grass_win_rate,f_dart_win_rate,f_win_rate,g_f_grass_win_rate,g_f_dart_win_rate,g_f_win_rate,m_grass_win_rate,m_dart_win_rate,m_win_rate
0,202202010608&rf,8,3歳以上1勝クラス,1,2022-06-26,12,?,1,1,2018100720,4,57.0,1030,?,?,?,10.2,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0.388889,0.0,0.388889,0.875,0.0,0.875,0.0,0.133333,0.125
1,202202010608&rf,8,3歳以上1勝クラス,1,2022-06-26,12,?,2,2,2019101236,3,54.0,1157,?,?,?,2.0,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0.000000,0.0,0.000000,0.000,0.0,0.000,0.0,0.000000,0.000
2,202202010608&rf,8,3歳以上1勝クラス,1,2022-06-26,12,?,3,3,2019103671,3,51.0,1200,?,?,?,32.4,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0.000000,0.0,0.000000,0.000,0.0,0.000,0.0,0.000000,0.000
3,202202010608&rf,8,3歳以上1勝クラス,1,2022-06-26,12,?,4,4,2019101536,3,54.0,1115,?,?,?,28.3,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0.000000,0.0,0.000000,0.000,0.0,0.000,0.0,0.000000,0.000
4,202202010608&rf,8,3歳以上1勝クラス,1,2022-06-26,12,?,5,5,2019105980,3,51.0,1177,?,?,?,13.3,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0.000000,0.0,0.000000,0.000,0.0,0.000,0.0,0.000000,0.000


add previous race

In [47]:
#delete unnecessary columns in horse_df
horse_df.drop(["father_id", "grandfather_id", "mother_id", "total_race_number"], axis = 1, inplace = True)

In [48]:
tmp_date_race_data = race_df[["date", "horse_id", "jockey_id"]]

In [49]:
#function 
def make_one_horse_data(tmp_date_horse_data, one_horse_df):
    one_horse_df = one_horse_df.sort_values('date', ascending = False) #False = 近いやつが上
    one_horse_df["same_jockey"] = (tmp_date_horse_data["jockey_id"] == one_horse_df["jockey_id"]) * 1.0
   
    
    one_horse_df = one_horse_df[(one_horse_df["date"] < tmp_date_horse_data["date"])] #レース前を抽出
    
    
    if (len(one_horse_df) <= 2):
        n = len(one_horse_df) 
    else:
        n = 3
    
    #print("one_horse_df")
    #print(one_horse_df)
    
    result_df = one_horse_df.copy()
    result_df.columns = result_df.columns + "_1"
    #print("result_df")
    #print(result_df)
    for i in range(1,n):
        shift_df = one_horse_df.shift(-i)
        #print("shift_df")
        #print(shift_df)
        shift_df.drop(["date"], axis = 1, inplace = True)
        shift_df.drop(["jockey_id"], axis = 1, inplace = True)
        shift_df.columns = shift_df.columns + "_" + str(i+1)
        result_df = pd.concat([result_df, shift_df], axis = 1)
        #print("result_df")
        #print(result_df)
    
    
    return result_df.head(1)                  #result_df.dropna()
        

In [50]:
one_data = tmp_date_race_data.iloc[0].copy()
one_horse_data = horse_df[horse_df["horse_id"] == "2018106606"].copy()
#print(one_data)
#print(one_horse_data)
tmp_race_df = make_one_horse_data(one_data, one_horse_data)
#print(tmp_race_df)

In [51]:
# うまくいくけど時間かかるやつ

for index, tmp in race_df.iterrows():
    tmp_race_df = tmp_race_df.append(make_one_horse_data(tmp, horse_df[horse_df["horse_id"] == tmp["horse_id"]]))


In [52]:
print(tmp_race_df.head(10))

       horse_id_1     date_1  whole_horse_number_1  odds_1 order_1  \
26778  2018106606 2022-06-11                     8    54.5       7   
6380   2018100720 2022-06-12                    14    33.2       4   
777    2019101236 2022-05-08                    13    26.3       6   
2440   2019103671 2022-06-11                    14    12.8      11   
961    2019101536 2022-04-09                    11    43.0       1   
4617   2019105980 2022-06-05                    16     3.7      14   
71118  2017103378 2022-06-11                    14    25.7       2   
13620  2018110132 2022-05-01                    12     6.9       5   
36657  2017104238 2021-09-26                    16    18.9       2   
3827   2019105218 2022-04-23                    16    44.4      12   

      jockey_id_1  burden_weight_1  race_distance_1  ground_condition_1  \
26778        1135             55.0             2000                   1   
6380         1030             57.0             1700                   1   
777 

In [53]:
# tmp_race_df.to_pickle("../data/csv/2021/tmp_race_12_26_df.pkl")
#tmp_race_df = pd.read_pickle("../data/csv/2021/tmp_race_12_25_df.pkl")

In [54]:
#delete horse1,2,3_id
tmp_race_df.drop(['horse_id_2','horse_id_3'], axis = 1, inplace = True)

In [55]:
#tmp_race_df = tmp_race_df.dropna()
tmp_race_df["whole_horse_number_2"] = tmp_race_df["whole_horse_number_2"].fillna(tmp_race_df["whole_horse_number_1"])
tmp_race_df["odds_2"] = tmp_race_df["odds_2"].fillna(tmp_race_df["odds_1"])
tmp_race_df["order_2"] = tmp_race_df["order_2"].fillna(tmp_race_df["order_1"])
tmp_race_df["burden_weight_2"] = tmp_race_df["burden_weight_2"].fillna(tmp_race_df["burden_weight_1"])
tmp_race_df["race_distance_2"] = tmp_race_df["race_distance_2"].fillna(tmp_race_df["race_distance_1"])
tmp_race_df["ground_condition_2"] = tmp_race_df["ground_condition_2"].fillna(tmp_race_df["ground_condition_1"])
tmp_race_df["order_2"] = tmp_race_df["order_2"].fillna(tmp_race_df["order_1"])
tmp_race_df["goal_time_2"] = tmp_race_df["goal_time_2"].fillna(tmp_race_df["goal_time_1"])
#tmp_race_df["velocity_2"] = tmp_race_df["velocity_2"].fillna(tmp_race_df["velocity_1"])
tmp_race_df["half_order_2"] = tmp_race_df["half_order_2"].fillna(tmp_race_df["half_order_1"])
tmp_race_df["last_time_2"] = tmp_race_df["last_time_2"].fillna(tmp_race_df["last_time_1"])
tmp_race_df["horse_weight_2"] = tmp_race_df["horse_weight_2"].fillna(tmp_race_df["horse_weight_1"])
tmp_race_df["weather_circumstance_小雨_2"] = tmp_race_df["weather_circumstance_小雨_2"].fillna(tmp_race_df["weather_circumstance_小雨_1"])
tmp_race_df["weather_circumstance_小雪_2"] = tmp_race_df["weather_circumstance_小雪_2"].fillna(tmp_race_df["weather_circumstance_小雪_1"])
tmp_race_df["weather_circumstance_晴_2"] = tmp_race_df["weather_circumstance_晴_2"].fillna(tmp_race_df["weather_circumstance_晴_1"])
tmp_race_df["weather_circumstance_曇_2"] = tmp_race_df["weather_circumstance_曇_2"].fillna(tmp_race_df["weather_circumstance_曇_1"])
tmp_race_df["weather_circumstance_雨_2"] = tmp_race_df["weather_circumstance_雨_2"].fillna(tmp_race_df["weather_circumstance_雨_1"])
tmp_race_df["weather_circumstance_雪_2"] = tmp_race_df["weather_circumstance_雪_2"].fillna(tmp_race_df["weather_circumstance_雪_1"])
tmp_race_df["main_place_その他_2"] = tmp_race_df["main_place_その他_2"].fillna(tmp_race_df["main_place_その他_1"])
tmp_race_df["main_place_中京_2"] = tmp_race_df["main_place_中京_2"].fillna(tmp_race_df["main_place_中京_1"])
tmp_race_df["main_place_中山_2"] = tmp_race_df["main_place_中山_2"].fillna(tmp_race_df["main_place_中山_1"])
tmp_race_df["main_place_京都_2"] = tmp_race_df["main_place_京都_2"].fillna(tmp_race_df["main_place_京都_1"])
tmp_race_df["main_place_函館_2"] = tmp_race_df["main_place_函館_2"].fillna(tmp_race_df["main_place_函館_1"])
tmp_race_df["main_place_小倉_2"] = tmp_race_df["main_place_小倉_2"].fillna(tmp_race_df["main_place_小倉_1"])
tmp_race_df["main_place_新潟_2"] = tmp_race_df["main_place_新潟_2"].fillna(tmp_race_df["main_place_新潟_1"])
tmp_race_df["main_place_札幌_2"] = tmp_race_df["main_place_札幌_2"].fillna(tmp_race_df["main_place_札幌_1"])
tmp_race_df["main_place_東京_2"] = tmp_race_df["main_place_東京_2"].fillna(tmp_race_df["main_place_東京_1"])
tmp_race_df["main_place_福島_2"] = tmp_race_df["main_place_福島_2"].fillna(tmp_race_df["main_place_福島_1"])
tmp_race_df["main_place_阪神_2"] = tmp_race_df["main_place_阪神_2"].fillna(tmp_race_df["main_place_阪神_1"])
tmp_race_df["race_rank_2"] = tmp_race_df["race_rank_2"].fillna(tmp_race_df["race_rank_1"])
tmp_race_df["ground_type_ダ_2"] = tmp_race_df["ground_type_ダ_2"].fillna(tmp_race_df["ground_type_ダ_1"])
tmp_race_df["ground_type_芝_2"] = tmp_race_df["ground_type_芝_2"].fillna(tmp_race_df["ground_type_芝_1"])
tmp_race_df["ground_type_障_2"] = tmp_race_df["ground_type_障_2"].fillna(tmp_race_df["ground_type_障_1"])
tmp_race_df["horse_weight_dif_2"] = tmp_race_df["horse_weight_dif_2"].fillna(tmp_race_df["horse_weight_dif_1"])
tmp_race_df["same_jockey_2"] = tmp_race_df["same_jockey_2"].fillna(tmp_race_df["same_jockey_1"])

tmp_race_df["whole_horse_number_3"] = tmp_race_df["whole_horse_number_3"].fillna(tmp_race_df["whole_horse_number_2"])
tmp_race_df["odds_3"] = tmp_race_df["odds_3"].fillna(tmp_race_df["odds_2"])
tmp_race_df["order_3"] = tmp_race_df["order_3"].fillna(tmp_race_df["order_2"])
tmp_race_df["burden_weight_3"] = tmp_race_df["burden_weight_3"].fillna(tmp_race_df["burden_weight_2"])
tmp_race_df["race_distance_3"] = tmp_race_df["race_distance_3"].fillna(tmp_race_df["race_distance_2"])
tmp_race_df["ground_condition_3"] = tmp_race_df["ground_condition_3"].fillna(tmp_race_df["ground_condition_2"])
tmp_race_df["order_3"] = tmp_race_df["order_3"].fillna(tmp_race_df["order_2"])
tmp_race_df["goal_time_3"] = tmp_race_df["goal_time_3"].fillna(tmp_race_df["goal_time_2"])
#tmp_race_df["velocity_3"] = tmp_race_df["velocity_3"].fillna(tmp_race_df["velocity_2"])
tmp_race_df["half_order_3"] = tmp_race_df["half_order_3"].fillna(tmp_race_df["half_order_2"])
tmp_race_df["last_time_3"] = tmp_race_df["last_time_3"].fillna(tmp_race_df["last_time_2"])
tmp_race_df["horse_weight_3"] = tmp_race_df["horse_weight_3"].fillna(tmp_race_df["horse_weight_2"])
tmp_race_df["weather_circumstance_小雨_3"] = tmp_race_df["weather_circumstance_小雨_3"].fillna(tmp_race_df["weather_circumstance_小雨_2"])
tmp_race_df["weather_circumstance_小雪_3"] = tmp_race_df["weather_circumstance_小雪_3"].fillna(tmp_race_df["weather_circumstance_小雪_2"])
tmp_race_df["weather_circumstance_晴_3"] = tmp_race_df["weather_circumstance_晴_3"].fillna(tmp_race_df["weather_circumstance_晴_2"])
tmp_race_df["weather_circumstance_曇_3"] = tmp_race_df["weather_circumstance_曇_3"].fillna(tmp_race_df["weather_circumstance_曇_2"])
tmp_race_df["weather_circumstance_雨_3"] = tmp_race_df["weather_circumstance_雨_3"].fillna(tmp_race_df["weather_circumstance_雨_2"])
tmp_race_df["weather_circumstance_雪_3"] = tmp_race_df["weather_circumstance_雪_3"].fillna(tmp_race_df["weather_circumstance_雪_2"])
tmp_race_df["main_place_その他_3"] = tmp_race_df["main_place_その他_3"].fillna(tmp_race_df["main_place_その他_2"])
tmp_race_df["main_place_中京_3"] = tmp_race_df["main_place_中京_3"].fillna(tmp_race_df["main_place_中京_2"])
tmp_race_df["main_place_中山_3"] = tmp_race_df["main_place_中山_3"].fillna(tmp_race_df["main_place_中山_2"])
tmp_race_df["main_place_京都_3"] = tmp_race_df["main_place_京都_3"].fillna(tmp_race_df["main_place_京都_2"])
tmp_race_df["main_place_函館_3"] = tmp_race_df["main_place_函館_3"].fillna(tmp_race_df["main_place_函館_2"])
tmp_race_df["main_place_小倉_3"] = tmp_race_df["main_place_小倉_3"].fillna(tmp_race_df["main_place_小倉_2"])
tmp_race_df["main_place_新潟_3"] = tmp_race_df["main_place_新潟_3"].fillna(tmp_race_df["main_place_新潟_2"])
tmp_race_df["main_place_札幌_3"] = tmp_race_df["main_place_札幌_3"].fillna(tmp_race_df["main_place_札幌_2"])
tmp_race_df["main_place_東京_3"] = tmp_race_df["main_place_東京_3"].fillna(tmp_race_df["main_place_東京_2"])
tmp_race_df["main_place_福島_3"] = tmp_race_df["main_place_福島_3"].fillna(tmp_race_df["main_place_福島_2"])
tmp_race_df["main_place_阪神_3"] = tmp_race_df["main_place_阪神_3"].fillna(tmp_race_df["main_place_阪神_2"])
tmp_race_df["race_rank_3"] = tmp_race_df["race_rank_3"].fillna(tmp_race_df["race_rank_2"])
tmp_race_df["ground_type_ダ_3"] = tmp_race_df["ground_type_ダ_3"].fillna(tmp_race_df["ground_type_ダ_2"])
tmp_race_df["ground_type_芝_3"] = tmp_race_df["ground_type_芝_3"].fillna(tmp_race_df["ground_type_芝_2"])
tmp_race_df["ground_type_障_3"] = tmp_race_df["ground_type_障_3"].fillna(tmp_race_df["ground_type_障_2"])
tmp_race_df["horse_weight_dif_3"] = tmp_race_df["horse_weight_dif_3"].fillna(tmp_race_df["horse_weight_dif_2"])
tmp_race_df["same_jockey_3"] = tmp_race_df["same_jockey_3"].fillna(tmp_race_df["same_jockey_2"])



In [56]:
print(tmp_race_df.shape)
tmp_race_df.isnull().sum()

(169, 102)


horse_id_1                   0
date_1                       0
whole_horse_number_1         0
odds_1                       0
order_1                      0
jockey_id_1                  0
burden_weight_1              0
race_distance_1              0
ground_condition_1           0
goal_time_1                  0
half_order_1                 0
last_time_1                  0
horse_weight_1               0
weather_circumstance_小雨_1    0
weather_circumstance_小雪_1    0
weather_circumstance_晴_1     0
weather_circumstance_曇_1     0
weather_circumstance_雨_1     0
weather_circumstance_雪_1     0
main_place_その他_1             0
main_place_中京_1              0
main_place_中山_1              0
main_place_京都_1              0
main_place_函館_1              0
main_place_小倉_1              0
main_place_新潟_1              0
main_place_札幌_1              0
main_place_東京_1              0
main_place_福島_1              0
main_place_阪神_1              0
race_rank_1                  0
ground_type_ダ_1              0
ground_t

In [57]:
race_df = pd.merge(race_df, tmp_race_df, left_on = ["horse_id"], right_on = ["horse_id_1"], how = "left")

same_jockey

In [58]:
same_jockey = pd.DataFrame(columns = ["same_jockey"])
same_jockey["same_jockey"] = (race_df["jockey_id_1"] == race_df["jockey_id"]) * 1.0 

race_df = pd.concat([race_df,same_jockey], axis = 1)
race_df.drop(["jockey_id"], axis = 1,inplace = True)

In [59]:
race_df.drop(['horse_id_1','date_1'], axis = 1, inplace = True)

In [60]:
race_df["horse_id"] = race_df["horse_id"].drop_duplicates()
race_df = race_df.dropna(subset = {"horse_id"})

In [61]:
print(race_df.shape)
race_df.head(5)

(168, 155)


,race_id,race_round,race_title,ground_condition,date,total_horse_number,order,frame_number,horse_number,horse_id,age,burden_weight,goal_time,half_order,last_time,odds,horse_weight,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_左,circle_右,circle_左,weather_circumstance_晴,weather_circumstance_小雨,weather_circumstance_雨,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雪,weather_circumstance_雪,place_函館,place_東京,place_阪神,札幌,中京,新潟,小倉,京都,福島,中山,sex_セ,sex_牝,sex_牡,f_grass_win_rate,f_dart_win_rate,f_win_rate,g_f_grass_win_rate,g_f_dart_win_rate,...,main_place_中京_2,main_place_中山_2,main_place_京都_2,main_place_函館_2,main_place_小倉_2,main_place_新潟_2,main_place_札幌_2,main_place_東京_2,main_place_福島_2,main_place_阪神_2,race_rank_2,ground_type_ダ_2,ground_type_芝_2,ground_type_障_2,horse_weight_dif_2,same_jockey_2,whole_horse_number_3,odds_3,order_3,burden_weight_3,race_distance_3,ground_condition_3,goal_time_3,half_order_3,last_time_3,horse_weight_3,weather_circumstance_小雨_3,weather_circumstance_小雪_3,weather_circumstance_晴_3,weather_circumstance_曇_3,weather_circumstance_雨_3,weather_circumstance_雪_3,main_place_その他_3,main_place_中京_3,main_place_中山_3,main_place_京都_3,main_place_函館_3,main_place_小倉_3,main_place_新潟_3,main_place_札幌_3,main_place_東京_3,main_place_福島_3,main_place_阪神_3,race_rank_3,ground_type_ダ_3,ground_type_芝_3,ground_type_障_3,horse_weight_dif_3,same_jockey_3,same_jockey
0,202202010608&rf,8,3歳以上1勝クラス,1,2022-06-26,12,?,1,1,2018100720,4,57.0,?,?,?,10.2,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0.388889,0.0,0.388889,0.875,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,6.0,0.0,14.0,5.2,5,54.0,1700.0,2.0,106.4,0.178571,39.2,474.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-2.0,1.0,1.0
1,202202010608&rf,8,3歳以上1勝クラス,1,2022-06-26,12,?,2,2,2019101236,3,54.0,?,?,?,2.0,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0.000000,0.0,0.000000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,0.0,16.0,45.5,7,56.0,1700.0,1.0,108.3,0.062500,40.2,506.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-8.0,0.0,0.0
2,202202010608&rf,8,3歳以上1勝クラス,1,2022-06-26,12,?,3,3,2019103671,3,51.0,?,?,?,32.4,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0.000000,0.0,0.000000,0.000,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-2.0,0.0,15.0,14.2,7,56.0,1800.0,1.0,118.2,0.216667,42.2,482.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,202202010608&rf,8,3歳以上1勝クラス,1,2022-06-26,12,?,4,4,2019101536,3,54.0,?,?,?,28.3,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0.000000,0.0,0.000000,0.000,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,1.0,13.0,28.1,2,56.0,1700.0,1.0,109.4,0.596154,38.7,514.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,1.0,1.0
4,202202010608&rf,8,3歳以上1勝クラス,1,2022-06-26,12,?,5,5,2019105980,3,51.0,?,?,?,13.3,?,0,1700,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0.000000,0.0,0.000000,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,-2.0,0.0,15.0,2.9,2,54.0,1200.0,3.0,70.5,0.066667,37.4,470.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-2.0,0.0,0.0


In [62]:
race_df.to_csv("../data/csv/2022/Jun_4/6_26_data.csv", index = False)